In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
import cv2
import os
import json
import random
from tqdm import tqdm
import shutil
from ensemble_boxes import *
import time

In [2]:
PROJECT_PATH = '...'
DATA_PATH = '...'
MODEL_PATH = '...'
SUBMISSION_PATH = PROJECT_PATH + 'submission_files/'
TEST_PATH1 = PROJECT_PATH + 'divided_test/test1/'
TEST_PATH2 = PROJECT_PATH + 'divided_test/test2/'
TEST_PATH3 = PROJECT_PATH + 'divided_test/test3/'

In [3]:
# wbf에 사용할 모델들
from ultralytics import YOLO
model1 = YOLO(MODEL_PATH + 'yolo_aug_e60_best.pt') # best
model2 = YOLO(MODEL_PATH + 'yolo_aug_e40_t0.62.pt') # second
model3 = YOLO(MODEL_PATH + 'yolon_aug_v3_e100_best.pt') # v3 best
model4 = YOLO(MODEL_PATH + 'yolon_aug_eq_e120_sgd_best.pt') # eq best
models = [model1, model2, model3, model4]

In [5]:
# 위의 모델들 결과 저장
def make_ensemble_predictions(data, models, conf_list):
    result = []
    for i in range(len(models)): 
        outputs = models[i].predict(source = data, save = False, conf = conf_list[i], device = 'cuda')
        result.append(outputs)
    return result

In [ ]:
# RAM 상의 이유로 테스트를 3개로 나눠서 진행
start = time.time()
results = make_ensemble_predictions(TEST_PATH1, models, conf_list = [0.75, 0.75, 0.75, 0.7])
# results = make_ensemble_predictions(TEST_PATH2, models, conf_list = [0.75, 0.75, 0.75, 0.7])
# results = make_ensemble_predictions(TEST_PATH3, models, conf_list = [0.75, 0.75, 0.75, 0.7])
print((time.time() - start) / 60)

In [7]:
# wbf
def wbf(predictions, weights, iou_thr = 0.6, skip_box_thr = 0.7):
    boxes_list = []
    scores_list = []
    labels_list = []

    for i in range(len(predictions[0])): # 이미지 수만큼
        box_list = []
        score_list = []
        label_list = []
        for pred in predictions:
            box_list.append(pred[i].boxes.xyxyn.tolist()) # box 결과 저장
            score_list.append(pred[i].boxes.conf.tolist()) # confidence score 결과 저장
            label_list.append(list(np.array(pred[i].boxes.cls.tolist(), dtype=int))) # 라벨 결과 저장

        boxes, scores, labels = weighted_boxes_fusion(box_list, score_list, label_list,
                                    weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        boxes_list.append(boxes)
        scores_list.append(scores)
        labels_list.append(labels)

    for box in boxes_list:
        for i in range(len(box)): 
            # 0과 1사이로 정규화되어 있기때문에 원본 크기로 복원
            box[i][0] = box[i][0] * 640
            box[i][1] = box[i][1] * 480
            box[i][2] = box[i][2] * 640
            box[i][3] = box[i][3] * 480

            # xmax, xmin을 w,h 형태로 바꾸기
            box[i][2] = box[i][2] - box[i][0]
            box[i][3] = box[i][3] - box[i][1]

    return boxes_list, score_list, labels_list


In [8]:
boxes_list, _, labels_list = wbf(results, weights = [2, 2, 1, 1], iou_thr = 0.6, skip_box_thr = 0.81)
print(len(boxes_list), len(labels_list))

14982 14982


In [9]:
img_id_dict = dict()

with open(PROJECT_PATH + 'test.json', "r") as f:
    label_coco = json.load(f)

for val in label_coco['images']:
    img_id_dict[val['file_name']] = val['id']

img_id_dict['test_0.png']

0

In [10]:
def make_submission_wbf(result, boxes_list, labels_list, file_name):
  output = result.copy()
  with open(PROJECT_PATH + 'test.json', 'r') as f:
      lab = json.load(f)
    
  # 객체가 탐지된 이미지
  cnt = 0
  for box in boxes_list:
      if len(box) != 0:
          cnt +=1
  print('탐지된 파일 수: ',cnt)

  annot = []

  for i in range(len(boxes_list)):
      if len(boxes_list[i]) != 0:
          for j in range(len(boxes_list[i])):
            result_dict = dict()
            result_dict['id'] = len(annot)
            result_dict['image_id'] = img_id_dict[output[i].path.split('/')[-1]]
            result_dict['category_id'] = int(labels_list[i][j])
            result_dict['bbox'] = list(boxes_list[i][j])
            result_dict['area'] = boxes_list[i][j][2] * boxes_list[i][j][3] # w * h
            annot.append(result_dict)
  
  lab['annotations'] = annot
  print('탐지된 객체 수: ', len(lab['annotations']))
  display(lab['annotations'][:3])
    
  with open(f'{SUBMISSION_PATH}{file_name}', "w") as json_file:
    json.dump(lab, json_file, ensure_ascii=False, indent=4)

  return lab['annotations']

In [11]:
# ann = make_submission_wbf(results[0], boxes_list, labels_list, 'yolov8n_wbf_best4_1_v6.json')
# ann = make_submission_wbf(results[0], boxes_list, labels_list, 'yolov8n_wbf_best4_2_v6.json')
ann = make_submission_wbf(results[0], boxes_list, labels_list, 'yolov8n_wbf_best4_3_v6.json')

탐지된 파일 수:  1185
탐지된 객체 수:  1262


[{'id': 0,
  'image_id': 123482,
  'category_id': 3,
  'bbox': [106.43628120422363,
   180.8892059326172,
   283.46606254577637,
   99.32742118835449],
  'area': 28155.952987088767},
 {'id': 1,
  'image_id': 123522,
  'category_id': 4,
  'bbox': [350.37330627441406,
   299.72371101379395,
   132.23289489746094,
   76.64148330688477],
  'area': 10134.5252069048},
 {'id': 2,
  'image_id': 123595,
  'category_id': 3,
  'bbox': [212.28622436523438,
   205.99611282348633,
   238.98834228515625,
   90.61952590942383],
  'area': 21657.010275759967}]

In [12]:
with open(SUBMISSION_PATH + 'yolov8n_wbf_best4_1_v6.json', 'r') as f:
    lab1 = json.load(f)

with open(SUBMISSION_PATH + 'yolov8n_wbf_best4_2_v6.json', 'r') as f:
    lab2 = json.load(f)
    
with open(SUBMISSION_PATH + 'yolov8n_wbf_best4_3_v6.json', 'r') as f:
    lab3 = json.load(f)

print(len(lab1['annotations']), len(lab2['annotations']), len(lab3['annotations']))

1023 1825 1262


In [13]:
concat_annot = lab1['annotations'] + lab2['annotations'] + lab3['annotations']

for idx, val in enumerate(concat_annot):
    val['id'] = idx # annotations id 0부터 다시 매기기

lab1['annotations'] = concat_annot
print('탐지한 전체 객체: ', len(lab1['annotations']))

display(lab1['annotations'][-3:])

탐지한 전체 객체:  4110


[{'id': 4107,
  'image_id': 33270,
  'category_id': 3,
  'bbox': [236.79275512695312,
   338.80788803100586,
   277.6530075073242,
   89.33526992797852],
  'area': 24804.206371981854},
 {'id': 4108,
  'image_id': 289,
  'category_id': 5,
  'bbox': [249.9590301513672,
   219.08798217773438,
   389.8860168457031,
   140.4356861114502],
  'area': 54753.91028098675},
 {'id': 4109,
  'image_id': 3227,
  'category_id': 3,
  'bbox': [0.0, 258.39929580688477, 543.4277725219727, 200.74029922485352],
  'area': 109087.85366315642}]

In [14]:
with open(SUBMISSION_PATH + 'yolov8n_wbf_best4_v6.json', 'w') as f:
    json.dump(lab1, f, ensure_ascii=False, indent=4)